In [1]:
# general settings
import pysesa
import os
import numpy as np
from laspy.file import File

#infile = os.path.expanduser("~")+os.sep+'pysesa_test'+os.sep+'example_100000pts.xyz' 
infile = r"D:\crs\proj\point_clouds\2016-09-21_Sandwich_v1_points-1-2.laz"

# =========================================================
def ascol( arr ):
    '''
    reshapes row matrix to be a column matrix (N,1).
    '''
    if len( arr.shape ) == 1: arr = arr.reshape( ( arr.shape[0], 1 ) )
    return arr

def scaled_x_dimension(las_file):
    x_dimension = las_file.X
    scale = las_file.header.scale[0]
    offset = las_file.header.offset[0]
    return(x_dimension*scale + offset)

def scaled_y_dimension(las_file):
    y_dimension = las_file.Y
    scale = las_file.header.scale[1]
    offset = las_file.header.offset[1]
    return(y_dimension*scale + offset)

def scaled_z_dimension(las_file):
    z_dimension = las_file.Z
    scale = las_file.header.scale[2]
    offset = las_file.header.offset[2]
    return(z_dimension*scale + offset)

X = []; Y = []; Z = []
R = []; G = []; B = []


inFile = File(infile, mode='r') 

X = scaled_x_dimension(inFile)
Y = scaled_y_dimension(inFile)
Z = scaled_z_dimension(inFile)
R = inFile.red
G = inFile.green
B = inFile.blue

# turn into single xyz array
coords = np.vstack((np.hstack(X), np.hstack(Y), np.hstack(Z))).T
del X, Y, Z

# turn into local coordinate system
coords = coords - np.min(coords, axis=0)

rgb = np.vstack((np.hstack(R), np.hstack(G), np.hstack(B))).T
del R, G, B

# scale the uint16 values to [0,1]
rgb = rgb/65535

## not going to plolt the whole point cloud - decimate by ...
decimation= 10

Using matplotlib backend: Qt4Agg


In [5]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.scatter(coords[::decimation,0], coords[::decimation,1], s=0.05, facecolors=rgb[::decimation,:], linewidth=0)
#plt.show()
plt.axis('tight')
plt.savefig('test.png', dpi=1000)
del fig, ax

In [ ]:
out = 0.5 #m output grid
detrend = 4 #ODR plane
proctype = 1 #Processing spectral parameters (no smoothing)
mxpts = 1024 # max pts per window
res = 0.05 #cm internal grid resolution
nbin = 20 #number of bins for spectral binning
lentype = 1 # l<0.5
taper = 1 # Hann taper
prc_overlap = 100 # 100% overlap between successive windows
minpts = 64 # min pts per window

pysesa.process(infile, out, detrend, proctype, mxpts, res, nbin, lentype, minpts, taper, prc_overlap)